In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from pathlib import Path

start_date = '2020-01-01'
end_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

df_tickers = pd.read_csv("/Users/marcomendieta/Documents/TFM/Data_stock/Tickets_indices.csv")
index_df = pd.read_csv('/Users/marcomendieta/Documents/TFM/Data_stock/stockindex.csv')

# Limpieza básica
df_tickers['ticker_market'] = df_tickers['ticker_market'].fillna("").astype(str).str.strip()
index_df['Sufijo'] = index_df['Sufijo'].fillna("").astype(str).str.strip()
index_df['País'] = index_df['País'].astype(str).str.strip()

# Ruta modificada
out_dir = Path("/Users/marcomendieta/Documents/TFM/Data_stock/StockData_csv")
out_dir.mkdir(parents=True, exist_ok=True)

# Ahora FECHA no estará en OUT_COLS
OUT_COLS = ["Ticker", "Year", "Close", "High", "Low", "Open", "Volume"]

def empty_row_for(ticker: str) -> pd.DataFrame:
    return pd.DataFrame([{
        "Ticker": ticker, "Year": pd.NA,
        "Close": pd.NA, "High": pd.NA, "Low": pd.NA, "Open": pd.NA, "Volume": pd.NA
    }])

for _, row in index_df.iterrows():
    sufijo = row['Sufijo']
    pais   = row['País']

    tickers = df_tickers[df_tickers['ticker_market'].str.endswith(sufijo)]['ticker_market'].tolist()
    all_rows = []

    if not tickers:
        pd.DataFrame(columns=OUT_COLS).to_csv(out_dir / f"stockprice_{pais.replace(' ', '_')}.csv", index=False)
        continue

    for t in tickers:
        t_clean = t.lstrip('$').strip()
        try:
            hist = yf.Ticker(t_clean).history(start=start_date, end=end_date, auto_adjust=False)
            if hist is None or hist.empty:
                all_rows.append(empty_row_for(t))
                continue

            hist = hist.reset_index()
            date_col = "Date" if "Date" in hist.columns else ("Datetime" if "Datetime" in hist.columns else None)
            if date_col is None:
                all_rows.append(empty_row_for(t))
                continue

            hist.rename(columns={date_col: "FECHA"}, inplace=True)

            if pd.api.types.is_datetime64tz_dtype(hist["FECHA"]):
                hist["FECHA"] = hist["FECHA"].dt.tz_convert(None)

            hist["FECHA"] = pd.to_datetime(hist["FECHA"])
            part = hist[["FECHA", "Open", "High", "Low", "Close", "Volume"]].copy()
            part.insert(0, "Ticker", t)
            part["Year"] = part["FECHA"].dt.year

            # Quitamos FECHA antes de guardar
            part = part.drop(columns=["FECHA"])
            part = part[OUT_COLS]
            all_rows.append(part)

        except Exception as e:
            all_rows.append(empty_row_for(t))

    stockprice = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame(columns=OUT_COLS)

    if not stockprice.empty:
        stockprice.drop_duplicates(subset=["Ticker", "Year"], keep="last", inplace=True)
        stockprice.sort_values(by=["Ticker", "Year"], inplace=True)

    stockprice.to_csv(out_dir / f"stockprice_{pais.replace(' ', '_')}.csv", index=False)


KeyError: 'ticker_market'